In [1]:
!pip install git+https://github.com/stasulam/faces.git@cnn

  Cloning https://github.com/stasulam/faces.git (to revision cnn) to /tmp/pip-req-build-pzoupk0f
Branch 'cnn' set up to track remote branch 'cnn' from 'origin'.
Switched to a new branch 'cnn'
  Running setup.py bdist_wheel for faces ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nudledx/wheels/c8/e3/b9/764bc63312620f531fea42459fa5d49d05ea226a097959ae37
Successfully built faces


Mount `Drive`. Images are stored in `Colab Notebooks` directory. We will use **global** variable, ie. `DIR` to manage access for `Dataset` class.

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Define `torch.transformers` which will be used in image preprocessing.

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

`FacesDataset` inherit methods from pytorch's `Dataset` in order to provide object, which will be used by `torch.utils.data.DataLoader`.

In [0]:
import os
from faces.preprocessing import FacesDataset

DIR = '/content/gdrive/My Drive/Colab Notebooks/'

# train set
trainset = FacesDataset(
    path_to_images=os.path.join(DIR, 'data/train/*/*.jpg'),
    transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=4,
    shuffle=4,
    num_workers=2
)
# test set
testset = FacesDataset(
    path_to_images=os.path.join(DIR, 'data/test/*/*.jpg'),
    transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=4,
    shuffle=False,
    num_workers=2
)

Define `LeNet` architecture suited for images of size `600x600`.

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class LeNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 147 * 147, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 147 * 147)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = LeNet()

In [6]:
import torch.optim as optim

classes = {
    'anna': torch.LongTensor([0]),
    'lukasz': torch.LongTensor([1])
}

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1):
    running_loss = 0.0
    for step, data in enumerate(trainloader, 0):
        inputs, labels = data
        labels = torch.cat([classes[label] for label in labels], 0)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if step % 50 == 49:
            print('[%d, %4d] loss: %.9f' %
                 (epoch + 1, step + 1, running_loss / 50))
            running_loss = 0.0
print('End!')

[1,   50] loss: 0.679658799
[1,  100] loss: 0.000026855
[1,  150] loss: 0.000008516
[1,  200] loss: 0.000018330
[1,  250] loss: 0.000008831
[1,  300] loss: 0.000003414
[1,  350] loss: 0.000013504
End!


In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = torch.cat([classes[label] for label in labels], 0)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (100 * correct / total))

Accuracy of the network: 100 %
